# Projeto 1 - Ciência dos Dados

Nome: Gustavo Katz

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [23]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as to_xlsx

In [24]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Gustavo\Documents\INSPER2\Cdados\Nova pasta\P1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [25]:
filename = 'PS5.xlsx'

In [26]:
train = pd.read_excel(filename).drop(columns=['0: Não relevante\n1: Relevante\n2: Muito relevante'])
train.head(5)

,Treinamento,Classificacao
0,"fala galera, livezinha no rainbow, vem ai dar ...",1
1,atualizações\n\nxbox series x : 10.3gb\nxbox o...,2
2,– season 6 resumo de tamanhos!\n\n– xbox: 26 g...,2
3,@loud_voltan vai jogar ps5 né veia? \n\nbom de...,1
4,@ocervejeiro_ ps5 pro com certeza. https://t.c...,1


In [27]:
test = pd.read_excel(filename, sheet_name = 'Teste').drop(columns=['0: Não relevante\n1: Relevante\n2: Muito relevante', 'Unnamed: 2'])
test.head(5)

,Teste,classificacao2
0,"@micalibruno apesar de ser um game de 2018, oc...",2
1,@ileohhh @fnstatusbr @brasil_fortnite no ps5 a...,0
2,rt @tiagohardco: ontem eu tava jogando pillars...,1
3,@heyzubshere pois é kfhskdsdfsdf e o ps5 tá da...,0
4,"@maiconkusterk sua namoradaa quebrar seu ps5, ...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Para poder classificaf os tweets como não relevantes, relevantes ou muito relevantes, foi utilizado um conceito dado por Dale Carnegie que explica apenas criticas não são uteis para se obter melhora. Com isso nossa categoria para não relevante foram as criticas negativas, para relevante foram as criticas positivas e para o muito relevante foram comparações com concorrentes

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [28]:
# Limpeza de caracteres desnecessários:
import re 

#função de limpeza removendo as pontuações
def cleanup(text):
    caracteres = '[!-.:,:?;/@")(|,[\n]' 
    reconstrucao = re.compile(caracteres)
    text_subbed = re.sub(reconstrucao, '', text)
    return text_subbed

In [29]:
#aqui temos são separadas as palavras de cada tweet de acordo com sua classificação e em sequência limpadas.

train0 = train.loc[(train['Classificacao'] == 0),:]
train1 = train.loc[(train['Classificacao'] == 1),:]
train2 = train.loc[(train['Classificacao'] == 2),:]
train0 = cleanup(str(list(train0['Treinamento']))).split()
train1 = cleanup(str(list(train1['Treinamento']))).split()
train2 = cleanup(str(list(train2['Treinamento']))).split()
   

In [30]:
#definindo as variaveis:
var0 = 1
var1 = 1
var2 = 1
lista0 = []
lista1 = []
lista2 = []


In [31]:
#definindo uma lista com todas as palavras sem repetição
sem_repeticao = []
resultado = []
for i in train0 + train1 + train2:
    if i not in sem_repeticao:
        sem_repeticao.append(i)
#print (len(sem_repeticao))

p_t0 = len(train0)
p_t1 = len(train1)
p_t2 = len(train2)
p_sem_repeticao = len(sem_repeticao)
#print(p_t0, p_t1, p_t2, p_sem_repeticao)


#limpeza de dados do teste com contas
for tweet in (test['Teste']):
    tweet = (tweet.split())
     
    tweet = ((cleanup(str(tweet))).split())
    for palavra in tweet:
        var0 = (train0.count(palavra) + 1)*var0 / (p_t0 + len(sem_repeticao))
        var1 = (train1.count(palavra) + 1)*var1 / (p_t1 + len(sem_repeticao))
        var2 = (train2.count(palavra) + 1)*var2 / (p_t2 + len(sem_repeticao))
        
    if var0 > var1 and var0 > var2:
        resultado.append(0)
    elif var1 > var0 and var1 > var2:    
        resultado.append(1)
    else:
        resultado.append(2)
        
    var0 = 1
    var2 = 1
    var1 = 1
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [32]:
test['resultado'] = resultado
test.head(10)

,Teste,classificacao2,resultado
0,"@micalibruno apesar de ser um game de 2018, oc...",2,2
1,@ileohhh @fnstatusbr @brasil_fortnite no ps5 a...,0,1
2,rt @tiagohardco: ontem eu tava jogando pillars...,1,0
3,@heyzubshere pois é kfhskdsdfsdf e o ps5 tá da...,0,2
4,"@maiconkusterk sua namoradaa quebrar seu ps5, ...",0,0
5,absurdo ps5 tá 6 mil,0,0
6,@engleooficial tomara que a sony lance um ps5 ...,0,0
7,"inclusive, queria poder fechar o jogo no ps5 d...",1,1
8,saturnorock @gabriell_medina @arielsis acho qu...,2,2
9,"🛠️ atualização liberada: ps5 (5,1gb)\n\n#fortn...",1,1


In [33]:
pd.crosstab(test['classificacao2'], test['resultado'], normalize=True)

resultado,0,1,2
classificacao2,,,
0,0.202572,0.057878,0.073955
1,0.112540,0.247588,0.093248
2,0.019293,0.009646,0.183280


___
### Concluindo

### Motivo para não utilizar novos dados do classificador para gerar novas amostras de treinamento:
Caso o classificador seja utilizado para novas amostras de treinamento, ao ocorrer um erro, esse erro poderá gerar ainda mais erros, fazendo com que a longo prazo a base de dados classificada acabe perdendo seu valor no algoritomo da conta.

### Diferentes cenários para Naïve Bayes:

Além de classificar a relevância de tweets, Naïve Bayes pode ser usado para detecção de fraude em transições financeiras, classificando entre "legal" ou "suspeito";  filtragem de spam; atribuição de temas em textos, como por exemplo: esportivo ou político. Naïve Bayes possui inúmeras aplacações, podendo ser utilizado até na  área de bioinformática, podendo identificar diferentes classes de proteínas

### Como aperfeiçoar Naïve Bayes:

https://www.vooo.pro/insights/6-passos-faceis-para-aprender-o-algoritmo-naive-bayes-com-o-codigo-em-python/#:~:text=Dicas%20para%20melhorar%20o%20poder%20do%20modelo%20Naive%20Bayes,-Aqui%20v%C3%A3o%20algumas&text=Se%20o%20conjunto%20de%20dados,conjunto%20de%20dados%20de%20teste:

https://machinelearningmastery.com/better-naive-bayes/

**/ este item foi encontrado em ambos os sites:**

A remoção de dados redundantes deve ser realizada ao classificar o que dever ser avaliado, pois pode causar exagero na sua importância

https://machinelearningmastery.com/better-naive-bayes/

As probabilidades são calculadas de forma independende, um dos motivos pelo qual ele possa ser chamado de “naive”, a partir dessa suposição o algoritmo pode-se acelerar a execução do algoritmo, calculando as probabilidades dos atributos em paralelo, isso pode ser feito utilizando CPUs, diferentes máquinas ou diferentes clusters.

https://datascience.stackexchange.com/questions/31413/how-to-improve-naive-bayes

Por quanto maior a base de dados, mais provavel é do resultado obtido estar correto, vale resaltar que ao utilizar o próprio classificador, ele fica sujeito a escalonar os erros, portanto não é recomendado.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**